## Demo for user story 105

The following scenario demonstrates the implementation of user story 105 for the RS Server Catalog.

The goal of this user story is to add a new search endpoint that can only search inside a collection,
so that the user can search easily for STAC items in a specific collections.

The following endpoints may now be called with an optional usage of the OwnerId parameter:
* GET /catalog/collections/{ownerId:collectionId}/search
* POST /catalog/collections/{ownerId:collectionId}/search

There is also a new fonction in the stac_client called "search_inside_collection" that can only search inside a collection.

### Initialisation

In [ ]:
# Init environment before running a demo notebook.
from resources.utils import *
init_demo()
from resources.utils import * # reload the global vars again

### Create a collection and 5 items to serve as a base for the demo

In [ ]:
# Create a test collection and stage test items
collection = create_test_collection()
items = stage_test_several_items()

In [ ]:
import requests

HEADERS={"x-api-key": apikey}

### Calling the endpoint GET /catalog/collections/{ownerId:collectionId}/search

Filter with projection epsg 4326

In [ ]:
import json

test_params = {"filter": "proj:epsg=4326"}

response = requests.get(f"{stac_client.href_catalog}/catalog/collections/{stac_client.owner_id}:{collection.id}/search", params=test_params, headers=HEADERS)
response.raise_for_status()

content = json.loads(response.content)
content

Filter with width 

In [ ]:
import json

test_params = {"filter": "width=2500"}

response = requests.get(f"{stac_client.href_catalog}/catalog/collections/{stac_client.owner_id}:{collection.id}/search", params=test_params, headers=HEADERS)
response.raise_for_status()

content = json.loads(response.content)
content

### Calling the endpoint POST /catalog/collections/{ownerId:collectionId}/search

Filter with width and projection epsg 3857

In [ ]:
WIDTH = 2500

parameters = {
            "filter-lang": "cql2-json",
            "filter": {
                "op": "and",
                "args": [
                    {"op": "=", "args": [{"property": "width"}, WIDTH]},
                    {"op": "=", "args": [{"property": "proj:epsg"}, 3857]},
                ],
            },
        }

response = requests.post(f"{stac_client.href_catalog}/catalog/collections/{stac_client.owner_id}:{collection.id}/search", json=parameters, headers=HEADERS)
response.raise_for_status()

content = json.loads(response.content)
content

### Using the new function "search_inside_collection" from the stac_client

Filter with ids

In [ ]:
HEIGHT = 3000

search = stac_client.search_inside_collection(owner_id=stac_client.owner_id, collection_id=collection.id, ids="DCS_01_S1A_20200105072204051312_ch1_DSDB_00002.raw")
results = list(search.items_as_dicts())
print(f"\nFound {len(results)} results")
for result in results:
    print(f"({result['collection']}) {result['id']}")



In [ ]:
HEIGHT = 3000

search = stac_client.search_inside_collection(owner_id=stac_client.owner_id, collection_id=collection.id, ids="DCS_01_S1A_20200105072204051312_ch1_DSDB_00002.raw")
results = list(search.items_as_dicts())
print(f"\nFound {len(results)} results")
for result in results:
    print(f"({result['collection']}) {result['id']}")

Filter with width and height using a filter

In [ ]:
filter = {
    "op": "and",
    "args": [
        {"op": "=", "args": [{"property": "width"}, WIDTH]},
        {"op": "=", "args": [{"property": "height"}, HEIGHT]},
    ],
}

search = stac_client.search(collections=[f"{stac_client.owner_id}_{collection.id}"], filter=filter)
results = list(search.items_as_dicts())
print(f"\nFound {len(results)} results")
for result in results:
    print(f"({result['collection']}) {result['id']}")